# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [ ]:
# Your code here; create the connection and cursor

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [3]:
# Your code here; use a subquery. No join is necessary
df = pd.DataFrame(cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                         FROM orders
                                         WHERE orderDate = "2003-01-31")''').fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

1


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [24]:
# Your code here
df = pd.DataFrame(cur.execute('''SELECT productName, SUM(quantityOrdered) AS totalOrdered,
                                 COUNT (orderNumber), productCode
                                 FROM products
                                 JOIN orderdetails
                                 USING (productCode)
                                 GROUP BY 1
                                 ORDER BY totalOrdered DESC''').fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.tail()

109


,productName,totalOrdered,COUNT (orderNumber),productCode
104,1999 Indy 500 Monte Carlo SS,855,25,S18_2870
105,1911 Ford Town Car,832,25,S18_2248
106,1936 Mercedes Benz 500k Roadster,824,25,S24_3969
107,1970 Chevy Chevelle SS 454,803,25,S24_1046
108,1957 Ford Thunderbird,767,24,S18_4933


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product


Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [28]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute('''SELECT COUNT (DISTINCT customerNumber) as numPurchaser, productName, productCode
               FROM products
               JOIN orderdetails
               USING (productCode)
               JOIN orders
               USING (orderNumber)
               GROUP BY 2
               ORDER BY 1 DESC''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()



109


,numPurchaser,productName,productCode
0,40,1992 Ferrari 360 Spider red,S18_3232
1,27,1934 Ford V8 Coupe,S18_2957
2,27,1952 Alpine Renault 1300,S10_1949
3,27,1972 Alfa Romeo GTA,S10_4757
4,27,Boeing X-32A JSF,S72_1253


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:
# Your code here
cur.execute('''SELECT employeeNumber, firstName, lastName, officeCode, O.city
               FROM employees E
               JOIN offices O
               USING (officeCode)
               JOIN customers C
               ON E.employeeNumber = C.salesRepEmployeeNumber
               JOIN orders
               USING (customerNumber)
               JOIN orderdetails
               USING (orderNumber)
               WHERE productCode IS (SELECT productCode 
                                     FROM products
                                     JOIN orderdetails
                                     USING (productCode)
                                     JOIN orders
                                     USING (orderNumber)
                                     GROUP BY 1
                                     HAVING COUNT(DISTINCT customerNumber) < 20);''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()



25


,employeeNumber,firstName,lastName,officeCode,city
0,1501,Larry,Bott,7,London
1,1166,Leslie,Thompson,1,San Francisco
2,1612,Peter,Marsh,6,Sydney
3,1611,Andy,Fixter,6,Sydney
4,1401,Pamela,Castillo,4,Paris


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [23]:
# Your code here
cur.execute('''SELECT employeeNumber, firstName, lastName, COUNT(customerNumber)
               FROM employees E
               JOIN customers C
               ON E.employeeNumber = C.salesRepEmployeeNumber
               GROUP BY salesRepEmployeeNumber
               HAVING AVG(creditLimit) > 15000''')

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()


15


,employeeNumber,firstName,lastName,COUNT(customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7


In [12]:
cur.execute('''SELECT salesRepEmployeeNumber, AVG(creditLimit) as averageCL
               FROM customers
               GROUP BY salesRepEmployeeNumber
               HAVING averageCL > 15000
               ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

15


,salesRepEmployeeNumber,averageCL
0,1165,100433.333333
1,1166,65266.666667
2,1188,73916.666667
3,1216,81533.333333
4,1286,66614.285714


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!